In [ ]:
import warnings
from pathlib import Path

import pandas as pd

data_dir = Path('../data')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def remove_institutions(
        pop_data: pd.DataFrame,
        institutions: pd.DataFrame,
        year,
):
    institutions = institutions.set_index(['district', 'plot_number'])
    institutions = institutions.loc[institutions.year == year]

    for idx in institutions.index:
        pop_data = pop_data.drop(index=idx, errors='ignore')

    return pop_data

def prepare_pop_data(
        population_data: pd.DataFrame,
        num_cols=None,
) -> pd.DataFrame:

    pop_frame = population_data.fillna(value=0)

    if not num_cols:
        num_cols = [
            'total_men',
            'total_women',
            'orthodox',
            'other_christian',
            'other_religion',
        ]

    pop_frame.loc[:, num_cols] = pop_frame.loc[:, num_cols].astype(int)

    pop_frame['lutheran'] = pop_frame['total_men total_women'.split()].sum(axis=1) \
                            - pop_frame['other_christian orthodox other_religion'.split()].sum(axis=1)

    pop_frame['total'] = pop_frame['other_christian orthodox other_religion lutheran'.split()].sum(axis=1)

    return pop_frame

In [ ]:
district_codes = pd.read_csv(data_dir / 'district_codes_1878.csv')
district_codes = {k: v for k, v in district_codes.itertuples(index=False)}
institutions = pd.read_csv(data_dir / 'institutions.csv', dtype={'district': str, 'plot_number': str})

districts_in_city = [
    'Valli',
    'Salakkalahti',
    'Repola',
    'Anina',
    'Papula',
    'P_Annan_kruunu',
    'Hiekka',
    'Pantsarlahti',
    'Viipurin_esikaupunki',
    'Paulovski',
    'Havi',
    'Saunalahti',
    'Pietarin_esikaupunki',
]

results_D = {}

for year in range(1880, 1921, 5):
    population_data = pd.read_csv(
            data_dir / 'interim' / f'pop_by_page_{year}.csv',
            index_col=0,
            dtype={'district': str, 'representative_plot': str, 'page_number': str},
        ).pipe(prepare_pop_data).pipe(remove_institutions, institutions, year)
    population_data = population_data[population_data.district.isin(districts_in_city)]
    population_data = population_data.loc[:, ['district', 'lutheran', 'orthodox', 'total']].groupby('district').sum()
    totals = pd.Series(population_data.sum(axis=0), name='totals')
    population_data = population_data.append(totals)

    print(f'    {year}:')
    print(population_data)
    print()

Model:

$ o_t = y_t - m_t $

$ m_t \sim Poisson(30) $

$ y_t = \theta y_{t-1} + \epsilon_t $

$ \epsilon_t \sim N(0, 5^2) $

$ \theta_t \sim N(0, 0.1^2) $

In [ ]:
import pandas as pd
import numpy as np
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt

data = pd.read_csv('data.csv', index_col=0)
data = data.loc[:, 'total orthodox income_per_person'.split()].dropna(axis=0)

with pm.Model() as model:
    mu_S = 0.3
    sigma_S = 0.1
    S = pm.Beta('S', mu=mu_S, sigma=sigma_S)
    mu_W = 2
    lambda_W = 1/mu_W
    W_k = pm.Exponential('W_k', lambda_W, observed=data.income_per_person)
    mu_P = 14
    P_k = pm.Poisson('P_k', mu_P, observed=data.total)
    mu_beta = np.array([0., 0., 0., 0.])
    sigma_beta = np.array(np.diagflat([0.25, 0.25, 0.25, 0.25]))
    beta = pm.MvNormal('beta', mu_beta, sigma_beta, shape=4)
    mu_O = np.exp(beta[0] + beta[1] * W_k + beta[2] * P_k + beta[3] * S)
    alpha_O = 2
    O_k = pm.Poisson('O_k', mu_O, observed=data.orthodox)

    prior = pm.sample_prior_predictive()
    posterior = pm.sample(init="adapt_diag")
    posterior_prediction = pm.sample_posterior_predictive(posterior)

az.plot_posterior(posterior)
az.plot_trace(posterior)
az.plot_forest(posterior, combined=False, hdi_prob=0.94)
pm.model_to_graphviz(model)
